Hi. Here are a few things I have learnt and found out about gpu optimization for ML training.

One of the main goals of gpu optimizatin is to maximize its utilization rate, since both buying gpu and keeping the powers on cost a lot of money. Below is a simple demo training script.

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset

device = "cuda" if torch.cuda.is_available() else "cpu"
X, y = torch.randn(10000, 64), torch.randn(10000, 1)
loader = DataLoader(TensorDataset(X, y), batch_size=256, shuffle=True)

model = nn.Linear(64, 1).to(device)
opt = torch.optim.SGD(model.parameters(), 0.01)
loss_fn = nn.MSELoss()

running_loss, running_count = 0.0, 0

for step, (xb, yb) in enumerate(loader, 1):
    xb, yb = xb.to(device), yb.to(device)
    loss = loss_fn(model(xb), yb)
    if torch.isnan(loss).any().item():
        print(f"NaN at step {step}"); break
    opt.zero_grad(); loss.backward(); opt.step()
    running_loss += loss.item() * xb.size(0)
    running_count += xb.size(0)

    if step % 50 == 0:
        print(f"avg_loss: {running_loss / running_count:.4f}")
        running_loss, running_count = 0.0, 0

Following are several ways this script can be optimized for gpu usage, mainly through cpu & gpu desyncronization.

In [ ]:
for step, (xb, yb) in enumerate(loader, 1):
    xb, yb = xb.to(device), yb.to(device)

Right now, every time gpu finishes processing a training point, it has to wait for the for-loop running on the cpu to push the next data point to its memory. A better way to do this is:

In [ ]:
loader = DataLoader(TensorDataset(X, y), batch_size=256, shuffle=True, pin_memory=True)
for step, (xb, yb) in enumerate(loader, 1):
    xb, yb = xb.to(device, non_blocking=True), yb.to(device, non_blocking=True)

pWhen the cpu loads the data into its memory, it pins them into several reserved physical addresses, so they won't move around due to other tasks running on the cpu. This allows the gpu to copy the data asynchronously by simply knowing the relevant physical memory addresses. non-blocking is set to True so that gpu can access the data itself without waiting for cpu's explicit allocation.

if torch.isnan(loss).any().item():
    print(f"NaN at step {step}"); break

The if-statement requires cpu execution, meaning that the gpu must send the loss value it has computed back to the cpu to check for NaN, causing a break in the gpu's training loop. This could be avoided by:

In [ ]:
nan_flag = torch.zeros(1, dtype=torch.bool, device=device)
nan_flag |= torch.isnan(loss).any()
if step % 50 == 0:
    if nan_flag.item(): print(f"NaN at step {step}"); break

Here, we initiate nan_flag as a 1 byte bool tensor set to 0 on the gpu. torch.isnan(loss).any() checks on the gpu if there is any NaN tensor and create a bool tensor. We can accumulate the result on the gpu by applying OR between the nan_flag and each bool tensor s.t. nan_flag will be set to 1/True if at least 1 NaN is dected. Then, we can check every 50 steps on the cpu if nan_flag is True.

The potential concern is the overhead from running all the extra training loops after a NaN is detected. Turns out the time saved from not having to copy between gpu's and cpu's memories is far greater than what it takes to complete a few extra loops.

In [ ]:
running_loss += loss.item() * xb.size(0)
running_count += xb.size(0)

Now it's cpu's turn to wait for the gpu to finish computing the losses. Although cpu idle time does not matter as much as gpu idle time, it would actually later result in gpu having to wait for the cpu to finish the computes here. Therefore, the best way to do this is to accumulate the loss metrics on the gpu.

In [ ]:
loss_sum = torch.zeros(1, device=device)
count = torch.zeros(1, device=device)
loss_sum += loss.detach() * xb.size(0); 
count += xb.size(0)

Similar to the nan_flag, here we initiate the loss and count vars as gpu tensors. Careful that loss should be detached if computed on gpu to to remove its computation graph from the memory to save some spaces.

Here is the complete script after the optimization. The key take away is to eliminate any bubble created from cpu & gpu attemps to syncronize through desyncronization. CUDA profilling tools like Nvidia Nsight is very useful for identifying the specific areas of code that can be optimized in this regard. 

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset

device = "cuda" if torch.cuda.is_available() else "cpu"
X, y = torch.randn(10000, 64), torch.randn(10000, 1)
loader = DataLoader(TensorDataset(X, y), batch_size=256, shuffle=True, pin_memory=True)

model = nn.Linear(64, 1).to(device)
opt = torch.optim.SGD(model.parameters(), 0.01)
loss_fn = nn.MSELoss()

loss_sum = torch.zeros(1, device=device)
count    = torch.zeros(1, device=device)
nan_flag = torch.zeros(1, dtype=torch.uint8, device=device)

for step, (xb, yb) in enumerate(loader, 1):
    xb, yb = xb.to(device, non_blocking=True), yb.to(device, non_blocking=True)
    loss = loss_fn(model(xb), yb)
    nan_flag |= torch.isnan(loss).any()
    opt.zero_grad(set_to_none=True); loss.backward(); opt.step()
    loss_sum += loss.detach() * xb.size(0); count += xb.size(0)

    if step % 50 == 0:
        if nan_flag.item(): print(f"NaN at step {step}"); break
        print(f"avg_loss: {(loss_sum/count).cpu().item():.4f}")
        loss_sum.zero_(); count.zero_()


The gpu itself also comes with many bottlenecks in areas such as compute, memory, and kernel scheduling. A kernel is a singular, containerized function that runs on a gpu with its allocated cores and memory. As such, having too many kernels will consumer a significant amount of memories. This can be avoided via kernel fusion either through torch.compile(), which is not optimized in many cases, or writting custom CUDA kernel in C++.

Below is my first attempt at writting a custom kernel, which add two arrays of floats, a common step for adding the bias vectors in a linear layer.

In [ ]:
#include <stdio.h>
#include <cuda_runtime.h>

// Kernel: C[i] = A[i] + B[i]
__global__ void vec_add(const float* A, const float* B, float* C, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        C[idx] = A[idx] + B[idx];
    }
}

int main() {
    int N = 10;
    size_t bytes = N * sizeof(float);

    // Host arrays
    float hA[10], hB[10], hC[10];
    for (int i = 0; i < N; i++) { hA[i] = i; hB[i] = i * 2; }

    // Device arrays
    float *dA, *dB, *dC;
    cudaMalloc(&dA, bytes);
    cudaMalloc(&dB, bytes);
    cudaMalloc(&dC, bytes);

    cudaMemcpy(dA, hA, bytes, cudaMemcpyHostToDevice);
    cudaMemcpy(dB, hB, bytes, cudaMemcpyHostToDevice);

    // Launch with 1 block of N threads
    vec_add<<<1, N>>>(dA, dB, dC, N);
    cudaDeviceSynchronize();

    cudaMemcpy(hC, dC, bytes, cudaMemcpyDeviceToHost);

    // Print result
    for (int i = 0; i < N; i++) {
        printf("%f + %f = %f\n", hA[i], hB[i], hC[i]);
    }

    cudaFree(dA); cudaFree(dB); cudaFree(dC);
    return 0;
}

CUDA also has a great feature called CUDA graph, which significanly reduces the overhead from kernel scheduling by dynamically generating modular, reusable template for instant scheduling replay. I will check it out later. Here is the link for anyone interested https://developer.nvidia.com/blog/cuda-graphs/.